To facilitate the reproduction of the final results of the contest, we have extracted some metadata in google drive. To execute this notebook without errors, please mount [this folder](https://drive.google.com/drive/folders/1wErskFiP_b9DVDFWUSdFBLon7eEx4AR9?usp=sharing) to this notebook using google colab **PRO**.

_Metadata we already generated using the method described in the paper or detailed instruction in extracting data tutorial [here]()._

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# RUN THIS CELL EVERY RESET RUNTIME
WORKDIR='/content/AIC2022-VER'
DATADIR='/content/AIC2022-VER/data' 
MOUNT_DATA_DIR='/content/drive/MyDrive/khoi/AIC22-VER/data-clean/' # replace this path

## Clone repo

In [ ]:
!pip install gdown==4.3.0

In [ ]:
!pip install --upgrade --no-cache-dir gdown google-cloud-storage python-Levenshtein webcolors

In [ ]:
!git clone https://github.com/nhtlongcs/AIC2022-VER/

Cloning into 'AIC2022-VER'...
remote: Enumerating objects: 2289, done.
remote: Counting objects: 100% (2289/2289), done.
remote: Compressing objects: 100% (1202/1202), done.
remote: Total 2289 (delta 1152), reused 2056 (delta 975), pack-reused 0
Receiving objects: 100% (2289/2289), 12.85 MiB | 31.47 MiB/s, done.
Resolving deltas: 100% (1152/1152), done.


In [ ]:
%cd $WORKDIR
!git checkout main
!git reset --hard HEAD
!git pull

/content/AIC2022-VER
Already on 'main'
Your branch is up to date with 'origin/main'.
HEAD is now at 97b459e Merge pull request #66 from nhtlongcs/guides-srl
Already up to date.


In [ ]:
!chmod +x $WORKDIR/tools/*

In [ ]:
%%capture
%cd $WORKDIR
!pip install faiss-gpu
!pip install -r requirements.txt
!pip install -e . 

In [ ]:
!python -m spacy download en_core_web_sm
!python -m nltk.downloader punkt wordnet omw-1.4

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


## Download checkpoints

In [ ]:
import gdown 
def download(token, output: str):
    url = f'https://drive.google.com/uc?id={token}'
    gdown.download(url, output, quiet=False)

In [ ]:
%cd $WORKDIR
download('1eylDX3tAJ2G0xhQsdOhU5uT1qH6SdzYK', 'retrieval-checkpoints.zip')
download('1VQ5vERytyTqDKlu2lM_ehk8BlgZ6wHgi', 'srl-checkpoints.zip')

/content/AIC2022-VER


Downloading...
From: https://drive.google.com/uc?id=1eylDX3tAJ2G0xhQsdOhU5uT1qH6SdzYK
To: /content/AIC2022-VER/retrieval-checkpoints.zip
100%|██████████| 10.0G/10.0G [01:22<00:00, 122MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VQ5vERytyTqDKlu2lM_ehk8BlgZ6wHgi
To: /content/AIC2022-VER/srl-checkpoints.zip
100%|██████████| 120M/120M [00:01<00:00, 114MB/s]


In [ ]:
%cp /content/drive/MyDrive/khoi/AIC22-VER/checkpoints/retrieval/checkpoints.zip $WORKDIR/retrieval-checkpoints.zip
%cp /content/drive/MyDrive/khoi/AIC22-VER/checkpoints/srl/srl-simple.zip $WORKDIR/srl-checkpoints.zip

In [ ]:
!unzip $WORKDIR/srl-checkpoints.zip
!unzip $WORKDIR/retrieval-checkpoints.zip
!rm -rf $WORKDIR/*.zip

Archive:  /content/AIC2022-VER/srl-checkpoints.zip
   creating: artifacts/
   creating: artifacts/vehicle/
   creating: artifacts/color/
  inflating: artifacts/vehicle/model.ckpt  
  inflating: artifacts/color/model.ckpt  
Archive:  /content/AIC2022-VER/retrieval-checkpoints.zip
   creating: artifacts/model-2xjz0ux8:v25/
  inflating: artifacts/model-2xjz0ux8:v25/model.ckpt  
   creating: artifacts/model-1ityga7d:v20/
  inflating: artifacts/model-1ityga7d:v20/model.ckpt  
   creating: artifacts/model-3e6jnmrm:v16/
  inflating: artifacts/model-3e6jnmrm:v16/model.ckpt  
   creating: artifacts/model-55pfbgaf:v15/
  inflating: artifacts/model-55pfbgaf:v15/model.ckpt  
   creating: artifacts/model-3r4fsrlp:v29/
  inflating: artifacts/model-3r4fsrlp:v29/model.ckpt  
   creating: artifacts/model-3kto46es:v7/
  inflating: artifacts/model-3kto46es:v7/model.ckpt  
   creating: artifacts/model-5h72x53e:v18/
  inflating: artifacts/model-5h72x53e:v18/model.ckpt  


In [ ]:
!ls /content/AIC2022-VER/artifacts/

color		    model-3e6jnmrm:v16	model-55pfbgaf:v15
model-1ityga7d:v20  model-3kto46es:v7	model-5h72x53e:v18
model-2xjz0ux8:v25  model-3r4fsrlp:v29	vehicle


## Prepare data

In [ ]:
%cd $DATADIR
%cp $MOUNT_DATA_DIR/AIC22_Track2_NL_Retrieval.zip $DATADIR/aic2022.zip
!unzip aic2022.zip -d AIC22_Track2_NL_Retrieval
!rm -rf aic2022.zip

In [48]:
!mkdir $DATADIR/meta/
%cp $DATADIR/AIC22_Track2_NL_Retrieval/*.json $DATADIR/meta/

In [ ]:
%%time
%cd $DATADIR
%cp $MOUNT_DATA_DIR/meta/meta_base.zip $DATADIR/meta_base.zip
!unzip -o $DATADIR/meta_base.zip -d $DATADIR/meta
!rm -rf $DATADIR/meta_base.zip

In [ ]:
%%time 
%cd $WORKDIR
%cp $MOUNT_DATA_DIR/meta/meta_motion_part.zip $DATADIR/meta_motion.zip
!unzip $DATADIR/meta_motion.zip -d $DATADIR/meta
!rm -rf $DATADIR/meta_motion.zip

In [ ]:
%%time 
%cd $WORKDIR
%cp $MOUNT_DATA_DIR/meta/meta_srl_part.zip $DATADIR/meta_srl_part.zip
!unzip -o $DATADIR/meta_srl_part.zip -d $DATADIR/meta
!rm -rf $DATADIR/meta_srl_part.zip

In [ ]:
%cd $WORKDIR
%cp $MOUNT_DATA_DIR/meta/meta_relation_part.zip $DATADIR/meta_relation.zip
!unzip $DATADIR/meta_relation.zip -d $DATADIR/
!rm -rf $DATADIR/meta_relation.zip

## Predict meta data

In [53]:
import os.path as osp

In [54]:
%cd $WORKDIR

track_json_path = osp.join(DATADIR,'meta/relation/neighbor_tracks.json')
!./tools/predict_srl.sh     configs/srl/col_infer.yml \
                            artifacts/color/model.ckpt \
                            configs/srl/veh_infer.yml \
                            artifacts/vehicle/model.ckpt \
                            $track_json_path \
                            $DATADIR/meta/extracted_frames \
                            ./results/classification/neighbors/

/content/AIC2022-VER
saving to folder ./results/classification/neighbors/
saving to ./results/classification/neighbors//color_prediction.json
saving to ./results/classification/neighbors//vehicle_prediction.json
Overriding configurating
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100% 759/759 [01:54<00:00,  6.62it/s]
Saved file to ./results/classification/neighbors//color_prediction.json
Overriding configurating
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficien

In [55]:
%cd $WORKDIR

track_json_path = osp.join(DATADIR,'meta/test_tracks.json')
!./tools/predict_srl.sh     configs/srl/col_infer.yml \
                            artifacts/color/model.ckpt \
                            configs/srl/veh_infer.yml \
                            artifacts/vehicle/model.ckpt \
                            $track_json_path \
                            $DATADIR/meta/extracted_frames \
                            ./results/classification/test_tracks/

/content/AIC2022-VER
saving to folder ./results/classification/test_tracks/
saving to ./results/classification/test_tracks//color_prediction.json
saving to ./results/classification/test_tracks//vehicle_prediction.json
Overriding configurating
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100% 46/46 [00:05<00:00,  8.20it/s]
Saved file to ./results/classification/test_tracks//color_prediction.json
Overriding configurating
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for ef

In [56]:
%cd $WORKDIR
!mkdir $DATADIR/meta/action
!python scripts/action/stop_turn_det.py -g $DATADIR/meta/train_tracks.json \
                                        -o $DATADIR/meta/action/train_stop_turn.json

!python scripts/action/stop_turn_det.py -g $DATADIR/meta/test_tracks.json \
                                        -o $DATADIR/meta/action/test_stop_turn.json

/content/AIC2022-VER
 63% 1363/2155 [00:03<00:02, 289.11it/s]/content/AIC2022-VER/external/detect_stop_turn/utils.py:61: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(vect_a, vect_b)/(np.linalg.norm(vect_a)*np.linalg.norm(vect_b)) #default: L2 norm
100% 2155/2155 [00:06<00:00, 313.61it/s]
Number of track which turns either left:  295
Number of track which turns either right:  393
Number of track which stops:  222
Number of straight-view track:  106
100% 184/184 [00:00<00:00, 187.45it/s]
Number of track which turns either left:  43
Number of track which turns either right:  45
Number of track which stops:  62
Number of straight-view track:  22


## Predict retrieval candidates

In [57]:
import gc
import os 
import json
import torch
import os.path as osp 

from opt import Opts
from src.predictor import Predictor
from src.models import MODEL_REGISTRY

In [58]:
config_data = {
    'model-55pfbgaf:v15': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'UTS-Best-0904'
    },
    'model-3kto46es:v7': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=224',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries_nlpaug_v2.jso')}n"
        ],
        'version': 'v2',
        'name': 'hcmussbj_eff3'
    },
    'model-3r4fsrlp:v29': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'UTS-Best-Second'
    },
    'model-5h72x53e:v18': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=256',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries_nlpaug_v2.json')}",
        ],
        'version': 'v2',
        'name': 'hcmussbj_eff3_best'
    },
    "model-2xjz0ux8:v25": {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=256',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries_nlpaug_v2.json')}",
        ],
        'version': 'v2',
        'name': 'hcmussbj_eff3'
    },
    'model-3e6jnmrm:v16': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'uts_eff3'
    },
    'model-1ityga7d:v20': {
        'params': [
            'extractors.img_encoder.args.version=3',
            'data.track.args.image_size=288',
            f"data.text.args.json_path={osp.join(DATADIR,'meta/test_queries.json')}",
        ],
        'version': 'v1',
        'name': 'uts_eff3'
    },
}

In [59]:
model_names = [
    'model-55pfbgaf:v15',
    'model-3kto46es:v7',
    'model-3r4fsrlp:v29',
    'model-5h72x53e:v18',
    "model-2xjz0ux8:v25",
    'model-3e6jnmrm:v16',
    'model-1ityga7d:v20',
]
data_params = [ f"data.track.args.json_path={osp.join(DATADIR,'meta/test_tracks.json')}",
                f"data.track.args.image_dir={osp.join(DATADIR,'meta/extracted_frames')}",
                f"data.text.args.shuffle=False",
                f"data.track.args.motion_path={osp.join(DATADIR,'meta/motion_map')}"
            ] 
model_name = model_names[0]
params = ['-o'] + config_data[model_name]['params'] + data_params
params

['-o',
 'extractors.img_encoder.args.version=3',
 'data.track.args.image_size=288',
 'data.text.args.json_path=/content/AIC2022-VER/data/meta/test_queries.json',
 'data.track.args.json_path=/content/AIC2022-VER/data/meta/test_tracks.json',
 'data.track.args.image_dir=/content/AIC2022-VER/data/meta/extracted_frames',
 'data.text.args.shuffle=False',
 'data.track.args.motion_path=/content/AIC2022-VER/data/meta/motion_map']

In [60]:
def run_predict(cfg, model, mode, bs, savedir, top_k=184, NUM_TTA=1):
    NUM_TTA = max(NUM_TTA, 1)
    for tta_id in range(NUM_TTA):
        savedir_tta = osp.join(savedir, str(tta_id))
        os.makedirs(savedir_tta, exist_ok=True)
        infer = Predictor(cfg=cfg, model=model, mode=mode, batch_size=bs, savedir=savedir_tta, top_k=top_k) # TOTAL NUMBER OF TRACKS (CANNOT SET -1)
        infer.predict()
        del infer
        gc.collect()
        torch.cuda.empty_cache()

In [61]:
%cd $WORKDIR
for i, model_name in enumerate(model_names):
    print(f'\n === running {model_name} {i+1}/{len(model_names)}=== \n')
    params = ['-o'] + config_data[model_name]['params'] + data_params
    savedir = osp.join(WORKDIR, f"results/retrieval/{model_name}")
    resume_ckpt = osp.join(WORKDIR, f"artifacts/{model_name}/model.ckpt")
    cfg_path = osp.join(WORKDIR,"configs/inference.yml") if config_data[model_name]['version'] == 'v1' else osp.join(WORKDIR,"configs/inferencev2.yml")
    cfg = Opts(cfg=cfg_path).parse_args(params)
    model = MODEL_REGISTRY.get(cfg.model["name"])(cfg)
    model = model.load_from_checkpoint(resume_ckpt, config=cfg, strict=False)
    run_predict(cfg, model, "complex", 32, savedir, top_k=184)

/content/AIC2022-VER

 === running model-55pfbgaf:v15 1/7=== 

Overriding configurating
Loaded pretrained weights for efficientnet-b3


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b3


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded pretrained weights for efficientnet-b3


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

Saved query_embeds.json to /content/AIC2022-VER/results/retrieval/model-1ityga7d:v20/0
Saved track_embeds.json to /content/AIC2022-VER/results/retrieval/model-1ityga7d:v20/0
Save query results to  /content/AIC2022-VER/results/retrieval/model-1ityga7d:v20/0/retrieval_results.json


In [62]:
save_dir = osp.join(WORKDIR, 'results/retrieval')
results = {}
for m in model_names:
    for idx in range(1): # TTA_IDS = [0]
      pred_json = osp.join(save_dir, m, str(idx), 'retrieval_results.json')      
      with open(pred_json, 'r') as f:
          data = json.load(f)
      for q, preds in data.items():
          if not(q in results.keys()):
              results[q] = {}
              results[q] = {}
          tmp = {}
          for i, k in enumerate(data[q]['pred_ids']):
              tmp[k] = data[q]['scores'][i]
          results[q][m] = tmp

# print(results.keys())
# print(data['1ed5b63a-0840-4fc3-8150-dd73b9b809ce']['pred_ids'])
q_ids = results.keys()
results_e = {}
for q in q_ids:
    results_e[q] = {}
    for model_name, model_res in results[q].items():
        for q_i in model_res.keys():
            if q_i in results_e[q]:
                results_e[q][q_i].append(1 * model_res[q_i])
            else:
                results_e[q][q_i] = [1 * model_res[q_i]]
# print(results_e['1ed5b63a-0840-4fc3-8150-dd73b9b809ce'])

for q in q_ids:
    for k in results_e[q]:
        results_e[q][k] = sum(results_e[q][k])
# print(results_e['1ed5b63a-0840-4fc3-8150-dd73b9b809ce'])

result_final = {}
for q in q_ids:
    result_final[q] = [(k, v) for k, v in results_e[q].items()]
    result_final[q] = sorted(result_final[q], key=lambda tup: tup[1], reverse=True)
    result_final[q] = [x[0] for x in result_final[q]]

# print(result_final['1ed5b63a-0840-4fc3-8150-dd73b9b809ce'])
result_path = osp.join(save_dir,'ensemble.json')
with open(result_path, 'w') as f:
    json.dump(result_final, f, indent=4)
print(f'saved prediction file at: {result_path}')

saved prediction file at: /content/AIC2022-VER/results/retrieval/ensemble.json


### Sanity check (have 184 queries)

In [63]:
import json
def load(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data
sub = load(result_path)
print(len(sub.keys()))
for k in sub.keys():
  if len(set(sub[k])) != 184:
      print(len(set(sub[k])))
print(sub[k])

184
['1d71dbf5-d375-49d5-8803-94c09660828d', '4dfef347-f354-4da1-b1a1-5e6be72e7788', '3a42701e-22ee-4b3f-a9c5-054fbce7037c', '0e7814b2-9d94-40c9-b221-a583b70a460e', 'b16df011-4464-4579-b996-5166e3c5f3f6', '96d82545-764a-4d39-8a92-f8d1dc3084ae', '18e09c74-6a9e-4432-9a2a-b28c85a34285', 'b87ce711-5ddd-4adf-ba8c-c72eb68e3f6f', 'bfccd19c-5971-4a24-9a2f-e6caead39751', '5b3851cb-5053-4b1d-8d95-37d1039e5bd7', '76cfa2ed-86f1-4b16-9167-a7e0a84b63fc', 'b908d536-4dd3-4a90-8ee5-9b23a270bbd8', '5cea7ea5-3e80-4e57-b15b-1043ffcf6092', 'd37e67c6-2ec2-40f8-ba6d-7d580887c76e', 'e1981f7e-dc53-488e-9301-9abd34aa33fb', '8c7f5240-9e2e-4c62-a1f4-d34f6ae5c25e', 'a7eb61e3-2eab-4e6e-8987-72886d99612e', 'af5d8758-a117-4c2c-a0d5-3a901f5e1746', 'ed81eaa0-6b0d-411f-91a9-667facde7800', '1159e0f6-c46c-443a-8ff6-b075b7e725de', 'df1651ab-8caa-45dd-b526-aa0cc86b7e83', '3713ed88-de22-4e55-8618-ff613c20d18c', 'e4c5ebc0-6d9f-4478-bcc6-fad1725dc5d2', '323c4be5-494a-4e64-9241-edbf6336676a', '92855130-9b90-45d1-84ef-a9c0736220

## Refinement

In [64]:
%cd $WORKDIR
!python scripts/action/stop_turn_det.py -g $DATADIR/meta/test_tracks.json \
                                        -o results/classification/test_tracks/action_prediction.json

/content/AIC2022-VER
100% 184/184 [00:00<00:00, 192.42it/s]
Number of track which turns either left:  43
Number of track which turns either right:  45
Number of track which stops:  62
Number of straight-view track:  22


In [ ]:
%cd $WORKDIR
!./tools/refine.sh  $WORKDIR/results/retrieval/ensemble.json \
                    $DATADIR/meta/srl/postproc/test_srl.csv \
                    $WORKDIR/results/test_relation_action_f1 \
                    $DATADIR/meta/relation/test_tracks_relation.json \
                    $DATADIR/meta/relation/neighbor_tracks.json \
                    $WORKDIR/results/classification/ \
                    $WORKDIR/results/

/content/AIC2022-VER
parse relation result and save to /content/AIC2022-VER/results/test_relation_action_f1
100% 184/184 [00:00<00:00, 623.59it/s]
run refinement, save result to /content/AIC2022-VER/results/
100% 184/184 [00:10<00:00, 17.44it/s]
n svo: 27
saved full class submission to: /content/AIC2022-VER/results/ensemble_full_Class.json
saved action only submission to: /content/AIC2022-VER/results/ensemble_action.json
saved vehcol only submission to: /content/AIC2022-VER/results/ensemble_vehcol.json
